In [15]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [16]:
cd '/content/drive/My Drive/apparels'

/content/drive/My Drive/apparels


In [0]:
!unzip test_ScVgIM0.zip


In [17]:
X_train.shape

(59400, 64, 64, 3)

In [3]:
!unzip train_LbELtWX.zip

Streaming output truncated to the last 5000 lines.
 extracting: train/45712.png         
 extracting: train/57389.png         
 extracting: train/5260.png          
 extracting: train/58492.png         
 extracting: train/58143.png         
 extracting: train/616.png           
 extracting: train/25467.png         
 extracting: train/37219.png         
 extracting: train/55151.png         
 extracting: train/57860.png         
 extracting: train/22627.png         
 extracting: train/52644.png         
 extracting: train/45879.png         
 extracting: train/33451.png         
 extracting: train/23972.png         
 extracting: train/38967.png         
 extracting: train/38753.png         
 extracting: train/33900.png         
 extracting: train/50011.png         
 extracting: train/1922.png          
 extracting: train/5638.png          
 extracting: train/37358.png         
 extracting: train/6609.png          
 extracting: train/6492.png          
 extracting: train/36174.png         

In [4]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import to_categorical
from keras.preprocessing import image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from tqdm import tqdm
import cv2

Using TensorFlow backend.


In [0]:
train=pd.read_csv('train.csv')

In [0]:
test=pd.read_csv('test.csv')

In [7]:
top=18
bottom=18
left=right=18

trainer_image=[]
for i in tqdm(range(train.shape[0])):
    img = image.load_img('train/'+train['id'][i].astype('str')+'.png', target_size=(28,28,3), grayscale=False)
    img1 = image.img_to_array(img)
    src=img1
    img1 = cv2.copyMakeBorder( src, top, bottom, left, right, borderType=cv2.BORDER_CONSTANT)
    trainer_image.append(img1)
X=np.array(trainer_image)    


100%|██████████| 60000/60000 [05:18<00:00, 188.31it/s]


In [0]:
del trainer_image

In [13]:
X.shape

(60000, 64, 64, 3)

In [0]:

y=train['label'].values
y = to_categorical(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.01)


In [0]:



from keras.layers import Dense
from keras import Model
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.applications.vgg16 import VGG16
vggmodel = VGG16(weights='imagenet', include_top=True)


553467904/553467096 [==============================] - 6s 0us/step


In [0]:
vggmodel.summary()


Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [0]:

def change_model(model, new_input_shape=(None, 64, 64, 3)):
    # replace input shape of first layer
    model._layers[0].batch_input_shape = new_input_shape

    # rebuild model architecture by exporting and importing via json
    new_model = keras.models.model_from_json(model.to_json())

    # copy weights from old model to new one
    for layer in new_model.layers:
        try:
            layer.set_weights(model.get_layer(name=layer.name).get_weights())
            print("Loaded layer {}".format(layer.name))
        except:
            print("Could not transfer weights for layer {}".format(layer.name))

    return new_model


In [0]:

new_model=change_model(vggmodel)
for layers in (new_model.layers)[:15]:
    print(layers)
    layers.trainable = False
X= new_model.layers[-2].output
predictions = Dense(10, activation="softmax")(X)
model_final = Model(input = new_model.input, output = predictions)

Loaded layer input_1
Loaded layer block1_conv1
Loaded layer block1_conv2
Loaded layer block1_pool
Loaded layer block2_conv1
Loaded layer block2_conv2
Loaded layer block2_pool
Loaded layer block3_conv1
Loaded layer block3_conv2
Loaded layer block3_conv3
Loaded layer block3_pool
Loaded layer block4_conv1
Loaded layer block4_conv2
Loaded layer block4_conv3
Loaded layer block4_pool
Loaded layer block5_conv1
Loaded layer block5_conv2
Loaded layer block5_conv3
Loaded layer block5_pool
Loaded layer flatten
Could not transfer weights for layer fc1
Loaded layer fc2
Loaded layer predictions


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  


In [0]:
opt = Adam(lr=0.0001)
model_final.compile(loss = keras.losses.categorical_crossentropy, optimizer = opt, metrics=["accuracy"])
model_final.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 64, 64, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 64, 64, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 64, 64, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 32, 32, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 32, 32, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 32, 32, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 16, 16, 128)       0   

In [0]:
model_final.fit(X_train, y_train, epochs=20, validation_data=(X_test, y_test))


Train on 48000 samples, validate on 12000 samples
Epoch 1/20
48000/48000 [==============================] - 144s 3ms/step - loss: 0.4395 - accuracy: 0.8412 - val_loss: 0.3977 - val_accuracy: 0.8558
Epoch 2/20
48000/48000 [==============================] - 137s 3ms/step - loss: 0.3104 - accuracy: 0.8871 - val_loss: 0.2932 - val_accuracy: 0.8907
Epoch 3/20
48000/48000 [==============================] - 138s 3ms/step - loss: 0.2725 - accuracy: 0.9003 - val_loss: 0.2961 - val_accuracy: 0.8950
Epoch 4/20
48000/48000 [==============================] - 137s 3ms/step - loss: 0.2457 - accuracy: 0.9088 - val_loss: 0.2921 - val_accuracy: 0.8997
Epoch 5/20
48000/48000 [==============================] - 137s 3ms/step - loss: 0.2269 - accuracy: 0.9161 - val_loss: 0.3071 - val_accuracy: 0.8988
Epoch 6/20
48000/48000 [==============================] - 137s 3ms/step - loss: 0.2084 - accuracy: 0.9224 - val_loss: 0.2610 - val_accuracy: 0.9111
Epoch 7/20
48000/48000 [==============================] - 137s

In [19]:

top=18
bottom=18
left=right=18

tester_image = []
for i in tqdm(range(test.shape[0])):
    img = image.load_img('test/'+test['id'][i].astype('str')+'.png', target_size=(28,28,1), grayscale=False)
    img2 = image.img_to_array(img)
    src=img2
    img2 = cv2.copyMakeBorder( src, top, bottom, left, right, borderType=cv2.BORDER_CONSTANT)
    tester_image.append(img2)
testing = np.array(tester_image)

100%|██████████| 10000/10000 [41:19<00:00,  4.03it/s]


In [0]:


prediction = model_final.predict(testing)





In [0]:
pred=np.argmax(prediction,axis=1)
test1=pd.read_csv('test.csv')
test1['label'] = pred


In [0]:
test1

,id,label
0,60001,9
1,60002,2
2,60003,1
3,60004,1
4,60005,0
...,...,...
9995,69996,9
9996,69997,1
9997,69998,8
9998,69999,1


In [0]:
from google.colab import files

test1.to_csv('test1.csv')
files.download('test1.csv')

In [0]:
X_train.shape

(59400, 64, 64, 3)

Without using prettrained vgg weights
(complete training of VGG16)[improved performance/accuracy]





In [0]:

new_model=change_model(vggmodel)

X= new_model.layers[-2].output
predictions = Dense(10, activation="softmax")(X)
model_final = Model(input = new_model.input, output = predictions)

Loaded layer input_1
Loaded layer block1_conv1
Loaded layer block1_conv2
Loaded layer block1_pool
Loaded layer block2_conv1
Loaded layer block2_conv2
Loaded layer block2_pool
Loaded layer block3_conv1
Loaded layer block3_conv2
Loaded layer block3_conv3
Loaded layer block3_pool
Loaded layer block4_conv1
Loaded layer block4_conv2
Loaded layer block4_conv3
Loaded layer block4_pool
Loaded layer block5_conv1
Loaded layer block5_conv2
Loaded layer block5_conv3
Loaded layer block5_pool
Loaded layer flatten
Could not transfer weights for layer fc1
Loaded layer fc2
Loaded layer predictions


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  


In [0]:
opt = Adam(lr=0.0001)
model_final.compile(loss = keras.losses.categorical_crossentropy, optimizer = opt, metrics=["accuracy"])
model_final.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 64, 64, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 64, 64, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 64, 64, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 32, 32, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 32, 32, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 32, 32, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 16, 16, 128)       0   

In [0]:
X_train.shape

(59400, 64, 64, 3)

In [0]:
checkpoint=keras.callbacks.ModelCheckpoint('apparels_vgg.h5', monitor='val_loss', verbose=0, save_best_only=False, save_weights_only=False, mode='auto', period=1)


In [0]:
model_final.fit(X_train, y_train, epochs=20, validation_data=(X_test, y_test),callbacks=[checkpoint])


model_final was trained for 16 epochs and then again for 5 more epochs

In [0]:
from keras.models import  load_model

model_final1 = load_model('apparels_vgg.h5')


In [18]:
model_final1.fit(X_train, y_train, epochs=5, validation_data=(X_test, y_test))


Train on 59400 samples, validate on 600 samples
Epoch 1/5
59400/59400 [==============================] - 1310s 22ms/step - loss: 0.0335 - accuracy: 0.9894 - val_loss: 0.2572 - val_accuracy: 0.9500
Epoch 2/5
59400/59400 [==============================] - 1395s 23ms/step - loss: 0.0370 - accuracy: 0.9897 - val_loss: 0.2964 - val_accuracy: 0.9483
Epoch 3/5
59400/59400 [==============================] - 1387s 23ms/step - loss: 0.0312 - accuracy: 0.9901 - val_loss: 0.4100 - val_accuracy: 0.9367
Epoch 4/5
59400/59400 [==============================] - 1364s 23ms/step - loss: 0.0235 - accuracy: 0.9928 - val_loss: 0.2765 - val_accuracy: 0.9583
Epoch 5/5
59400/59400 [==============================] - 1379s 23ms/step - loss: 0.0237 - accuracy: 0.9929 - val_loss: 0.3462 - val_accuracy: 0.9367


In [20]:
testing.shape

(10000, 64, 64, 3)

In [0]:
prediction = model_final1.predict(testing)


In [0]:
pred=np.argmax(prediction,axis=1)
test2=pd.read_csv('test.csv')
test2['label'] = pred

In [0]:
from google.colab import files

test2.to_csv('test2.csv')
files.download('test2.csv')